# Notes


Cookie: standard


### `{{cookiecutter.project_name}}::{{cookiecutter.session_id}}`


## 1. Usage


### 1.1. Jupyter


_You can fill inn the MarkDown cells (the cells without "numbering") by double-clicking them. Also remember, press `shift + enter` to execute a cell._

A couple of useful links:

- [How to write MarkDown](https://github.com/adam-p/markdown-here/wiki/Markdown-Cheatsheet#hr)
- [Jupyter notebooks](https://jupyter.org/)
- [cellpy](https://cellpy.readthedocs.io/en/latest/)


### 1.2. The `cellpy` notebooks


This set of notebooks is intended to give you a structure for performing a proper `cellpy` analysis of your data utilising the `cellpy.utils.bathc` tool.

An example of a project structure can be this:

```bash

cellpy_project/
└── experiment_001
    ├── batch_file.json
    ├── data
    │   ├── external
    │   ├── interim
    │   ├── processed
    │   └── raw
    ├── out
    │   └── note.md
    ├── notebook: 00_experiment_001_notes.ipynb
    ├── notebook: 01_experiment_001_loader.ipynb
    ├── notebook: 02_experiment_001_life.ipynb
    ├── notebook: 03_experiment_001_cycles.ipynb
    ├── notebook: 04_experiment_001_ica.ipynb
    └── notebook: 05_experiment_001_plots.ipynb

```

The `00_experiment_001_notes.ipynb` notebook can be used for writing a log, as well as giving you some information on how to do stuff.

The `01_experiment_001_loader.ipynb` notebook is the one you use to load the data from the tester(s) and create cellpy-files. You should run this first. There are several options on how to store the cellpy-files and the journal, so chose the option that suits you best. You only have to re-run this notebook if your data needs to be updated.

The next three notebooks use the cellpy-files produced by `01_experiment_001_loader.ipynb`. Try to keep them "understandable" and well structured so that the future you dont get totally lost trying to re-run them.

The last notebook is ment to contain important plots that you would like to be able to use later on. They should preferably be "stand-alone" and not rely on running the other notebooks (as a helpful gesture to the future you).


## 2. Key information about the current experiment


**Experimental-id:** `{{cookiecutter.notebook_name}}`  
**Short-name:** `{{cookiecutter.session_id}}`  
**Project:** `{{cookiecutter.project_name}}`  
**By:** `{{cookiecutter.author_name}}`  
**Date:** `{{cookiecutter.date}}`


### Notebooks

- notes and information [link](00_{{cookiecutter.notebook_name}}_notes.ipynb)
- processing raw data [link](01_{{cookiecutter.notebook_name}}_loader.ipynb)
- life [link](02_{{cookiecutter.notebook_name}}_life.ipynb)
- cycles [link](03_{{cookiecutter.notebook_name}}_cycles.ipynb)
- ica [link](04_{{cookiecutter.notebook_name}}_ica.ipynb)
- plots [link](05_{{cookiecutter.notebook_name}}_plots.ipynb)


## 3. Short summary of the experiment before processing

It is often helpful to formulate what you wanted to achieve with your experiment before actually going into depth of the data. I believe that it does not make you "biased" when processing your data, but instead sharpens your mind and motivates you to look more closely on your results. I might be wrong, off course. Then just skip filling in this part.


### Main purpose

(_State the main hypothesis for the current set of experiment_)


### Expected outcome

(_What do you expect to find out? What kind of tests did you perform?_)


### Special considerations

(_State if there are any special considerations for this experiment_)


## 4. Log

(_Here you should fill inn what you have done during your analysis_)


## 5. Summary

It is always helpful to formulate what you learned from your experiment. It is very unlikely that I am wrong here. So, please, don't skip filling in this part.


(_What did you discover?_)

(_What should you do further?_)


## Appendix A - Tips and tricks


### Tip: Alternative methodology for creating batch and loading data first time

If you want more control, you can perform each step seperately. Instead
of using the `batch.load` method, use:

```python

# Parameters for the batch
project = "{{cookiecutter.project_name}}"
name = "{{cookiecutter.session_id}}"
batch_col = "b01"  # edit this if you are not using the standard batch column

# Create the batch object
b = batch.init(name, project, batch_col=batch_col)

## Setting some prms if default values are not OK for you
b.experiment.export_raw = False
b.experiment.export_cycles = False
b.experiment.export_ica = False
# b.experiment.force_cellpy = True  # force to select the cellpy-files
# b.experiment.force_raw = True  # force to select the raw files
# b.experiment.all_in_memory = True  # store all data in memory, defaults to False
# b.save_cellpy = False  # prevent saving the generated cellpy-files
# b.last_cycle = 1000  # only load max 1000 cycles

# load info from your db and write the journal pages
b.create_journal()
b.duplicate_journal()

# Create the apropriate folders
b.paginate()
```


### Tip: Alternative methodology for creating batch and loading data from journal file

If you want more control, you can perform each step seperately. Instead
of using the `batch.load` method, use:

```python

cellpy_batch_file = "cellpy_batch_{{cookiecutter.session_id}}.json"
b = batch.from_journal(cellpy_batch_file)
b.link()
# or b.update() if you would like to get new data from the tester.

```


### Trick: Use magic to automatically reload modules

This allows you to modify modules (and scripts) in another editor and see the changes within Jupyter without shutting down the kernel.


```python
# in the first notebook cell
%load_ext autoreload
%autoreload 2
```


### Example: Find files if automatic search fails


```python
def add_files_to_pages(p):
    raw_folder = pathlib.Path(prms.Paths.rawdatadir)
    rf = []
    for f in p.index:
        print(f, end=": ")
        new_rf = list(raw_folder.glob(f"{f}*.res"))
        print(new_rf)
        rf.append(new_rf)
    p.raw_file_names = rf
    return p

b.pages = add_files_to_pages(b.pages)
```


### Example: Select first cycle from all cells and save them as a csv-file with headers


```python
import pathlib
import csv
from itertools import zip_longest

# Creating "csv-save-able" structure
header1 = []
header2 = []
out = []
cycle = 1

for label in cell_labels:
    print(label, end="")
    try:
        c = b.experiment.data[label]
        sss = c.get_cap(cycle=cycle, interpolated=True, dx=0.01)
        cap = sss.capacity
        vlt = sss.voltage
        out.append(cap)
        out.append(vlt)
        header1.append(label)
        header1.append(label)
        header2.append("capacity")
        header2.append("voltage")
        print(" - OK")
    except:
        print(" - could not process")

# Saving to csv (list of lists)
filename = "data/processed/first_voltage_curve_for_all_cells.csv"

with open(filename,"w+") as f:
    writer = csv.writer(f, delimiter=';')
    writer.writerow(header1)
    writer.writerow(header2)
    for values in zip_longest(*out):
        writer.writerow(values)

```


### Example: Film-plot of the evolution of dQ/dV peaks


```python
import holoviews as hv

import cellpy
from cellpy.utils import ica

c = cellpy.get("a/cellpy/file.h5")


first_cycle = 2
last_cycle = 30

cycle = list(range(first_cycle,last_cycle+1))
ica_c, ica_dc = ica.dqdv_frames(c,cycle=cycle, voltage_resolution=0.005, normalizing_factor=1, split=True, tidy=False)

voltages_c = ica_c.voltage.values.ravel()
dqs_c = ica_c.drop("voltage", axis=1)
cycles_c = dqs_c.columns.get_level_values(0).values.ravel().astype('int32')
dqs_c = dqs_c.values.T

voltages_dc = ica_dc.voltage.values.ravel()
dqs_dc = ica_dc.drop("voltage", axis=1)
cycles_dc = dqs_dc.columns.get_level_values(0).values.ravel().astype('int32')
dqs_dc = -dqs_dc.values.T

options = {
    "xlabel": "voltage",
    "ylabel": "cycle number",
    "xlim": (0.2, 0.65),
}
ica_im_c = hv.Image((voltages_c, cycles_c, dqs_c)).opts(title=f"charge", **options)
ica_im_dc = hv.Image((voltages_dc, cycles_dc, dqs_dc)).opts(title=f"discharge", **options)

ica_im_c + ica_im_dc

```


### Example: use `ipydatagrid` to edit `b.pages` (the journal pages)


#### Install if missing:

```python

!pip install ipydatagrid

```

(you proably have to restart your jupyter session for it to load the js)


#### Import and create the interactive datagrid

```python
from ipydatagrid import DataGrid

dg = DataGrid(b.pages, editable=True)
```


#### Show it and edit it

```python
dg
```


#### Update the journal pages

```python
b.pages = dg.data

## Don't forget to save it if you want to keep your changes:
# b.save_journal()
# b.duplicate_journal()

```


### Example: collect data using `collectors`, but plot with another tool (`seaborn`)

```python

# Setting up the collector without running it (autorun=False):
cap_summaries = collectors.BatchSummaryCollector(
    b,
    autorun=False,
    normalize_cycles=True,
    fig_title="Areal Capacity Plots",
    data_collector_arguments={
        "columns": [
            "charge_capacity_areal",
            "discharge_capacity_areal"
        ],
    }
)

# Collecting data without plotting:
cap_summaries.update(update_data=True, update_plot=False)


df = cap_summaries.data.copy()

# Best to convert to long format when using seaborn
#  trick to decide on id and value variables:
cols = df.columns.to_list()
possible_id_vars = ["cycle", "equivalent_cycle", "cell", "group", "sub_group"]
id_vars = [v for v in possible_id_vars if v in cols]
value_vars = [v for v in cols if v not in id_vars]
# doing the melting from (rather) wide to long format:
df = df.melt(value_vars=value_vars, id_vars=id_vars, var_name="variable", value_name="value")

# Plotting
fig = sns.relplot(data=df, x="cycle", y="value", row="variable", hue="group", style="sub_group", palette="tab10", height=3, aspect=2)
```
